In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

busy = pd.read_parquet("../processed_data/london_busy.parquet")
busy

,name,group,geometry,latitude,longitude
0,Tesco Express,shop,b'\x01\x01\x00\x00\x00tb\x0f\xedc\x05\xc0\xbf(...,51.511122,-0.125165
1,None,shop,b'\x01\x01\x00\x00\x00H\xb5\x99T~c\xc7\xbf*\x0...,51.550055,-0.182724
2,Tesco Express,shop,b'\x01\x01\x00\x00\x00\xee\xc0\xef\x95/\x1e\xc...,51.562078,-0.149359
3,Tesco Express,shop,"b'\x01\x01\x00\x00\x00""\xf7\x19\xbcT\x11\xc1\x...",51.545341,-0.133341
4,Chris Dry Cleaners,shop,b'\x01\x01\x00\x00\x00\x9d~\xf5]\xb6B\xb3\xbf\...,51.551313,-0.075237
...,...,...,...,...,...
91888,None,parking,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0b\x00...,51.515215,-0.168673
91889,None,parking,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\t\x00\x...,51.413759,-0.077673
91890,None,parking,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x08\x00...,51.494106,-0.236990
91891,None,parking,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x0c\x00...,51.409700,-0.149445


In [3]:
# Count total rows
initial_rows = len(busy)
print(initial_rows, 'total number of initial rows')

# Missing Longitude analysis
missing_long = busy[busy['longitude'].isna()]
num_missing_long = len(missing_long)
print(num_missing_long, 'number of rows that have Longitude missing (unidentifiable)')
print(num_missing_long / initial_rows * 100, '% percentage of rows that are missing Longitude')

# Rows with Longitude present
print(initial_rows - num_missing_long, 'number of rows that contain Longitude')

# Missing Latitude analysis
missing_lat = busy[busy['latitude'].isna()]
num_missing_lat = len(missing_lat)
print(num_missing_lat, 'number of rows that have Latitude missing (unidentifiable)')
print(num_missing_lat / initial_rows * 100, '% percentage of rows that are missing Latitude')

# Rows with Latitude present
print(initial_rows - num_missing_lat, 'number of rows that contain Latitude')

# Check rows missing both Latitude and Longitude
missing_both = busy[busy['latitude'].isna() & busy['longitude'].isna()]
num_missing_both = len(missing_both)
print(num_missing_both, 'number of rows missing both Latitude and Longitude')

# Verify if all missing values are in the same rows
if num_missing_both == num_missing_lat == num_missing_long:
    print("✅ All missing Latitude and Longitude values are from the same rows.")
else:
    print("⚠️ Latitude and Longitude missing values are not from the exact same rows.")


91893 total number of initial rows
0 number of rows that have Longitude missing (unidentifiable)
0.0 % percentage of rows that are missing Longitude
91893 number of rows that contain Longitude
0 number of rows that have Latitude missing (unidentifiable)
0.0 % percentage of rows that are missing Latitude
91893 number of rows that contain Latitude
0 number of rows missing both Latitude and Longitude
✅ All missing Latitude and Longitude values are from the same rows.


In [5]:
# Missing name analysis
missing_name = busy[busy['name'].isna()]
num_missing_name = len(missing_name)
print(num_missing_name, 'number of rows that have name missing')
print(num_missing_name / initial_rows * 100, '% percentage of rows that are missing name')

# Rows with Latitude present
print(initial_rows - num_missing_name, 'number of rows that contain name')

20613 number of rows that have name missing
22.431523619862233 % percentage of rows that are missing name
71280 number of rows that contain name
